In [66]:
#!pip install urllib3
#!pip install bs4

# For data manipulation
import pandas as pd
from urllib.request import urlopen, Request
import numpy as np
from datetime import datetime

# To extract fundamental data
from bs4 import BeautifulSoup

In [67]:
def transform_to_millions(value):
    if pd.isna(value) or value == '-':
        return np.nan
    suffix = value[-1]  # Get the last character (M, B, etc.)
    num = float(value[:-1])  # Get the numeric part
    if suffix == 'M':
        return num * 1e6  # Convert to millions
    elif suffix == 'B':
        return num * 1e9  # Convert to billions
    else:
        return num  # Return as is if no suffix

# Functions to Parse Data from FinViz

# Initialize Pandas DataFrame to Store the Data

In [68]:
date = datetime.today().strftime('%Y-%m-%d')
output_directory = "output/"
input_directory = "input/"
csv_name='data'+date
df = pd.read_csv(input_directory+csv_name+".csv",index_col=0)
df.head(10)

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own,EPS next Y,Income,Perf Year,Perf YTD,Profit Margin,Payout,Sales
ELTK,2.73,14.58,9.61,-,0.09,0.70,-,19.50%,9.40%,146.80%,25.30%,1.07,4.10M,143.60%,149.86%,10.00%,-,41.40M
OPCH,4.09,34.89,26.84,3.49,0.79,0.87,-,11.90%,7.60%,28.80%,0.10%,1.14,159.50M,9.72%,0.90%,3.90%,0.00%,4.04B
MELI,30.14,100.03,50.09,2.02,2.37,12.23,-,34.90%,10.40%,204.20%,7.85%,24.43,618.00M,80.54%,43.51%,5.50%,0.00%,11.33B
KOPN,5.21,-,-,-,0.00,-0.22,-,-61.90%,-90.60%,-64.30%,0.90%,-0.07,-20.60M,80.40%,64.40%,-44.20%,-,46.60M
IMBIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRMD,1.28,2.97,3.91,-,-,8.63,-,-,-,1376.90%,0.46%,6.57,-,90.60%,-12.38%,-,-,1.62B
ABIO,0.75,-,-,-,0.00,-0.56,-,-18.80%,-,58.90%,18.50%,-,-8.00M,-9.25%,-13.08%,-,-,-
BHFAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
LBPH,2.20,-,-,-,0.00,-2.56,-,-63.50%,-,0.10%,21.06%,-1.83,-45.80M,83.10%,135.89%,-,-,-
IZM,3.22,15.38,-,-,1.16,0.25,-,31.10%,11.40%,334.20%,-,-,2.60M,-,19.94%,0.90%,0.00%,290.40M


# Data Clearning: Further Parse the Data into Numeric Types
Remove % Sign and Convert Values to Numeric Type

In [69]:
df['Dividend %'] = df['Dividend %'].str.replace('%', '')
df['ROE'] = df['ROE'].str.replace('%', '')
df['ROI'] = df['ROI'].str.replace('%', '')
df['EPS Q/Q'] = df['EPS Q/Q'].str.replace('%', '')
df['EPS next Y'] = df['EPS next Y'].str.replace('%', '')
df['Insider Own'] = df['Insider Own'].str.replace('%', '')
df['Perf YTD'] = df['Perf YTD'].str.replace('%', '')
df['Perf Year'] = df['Perf Year'].str.replace('%', '')
df['Income'] = df['Income'].apply(lambda x: transform_to_millions(x))
df['Profit Margin'] = df['Profit Margin'].str.replace('%', '')
df = df.apply(pd.to_numeric, errors='coerce')
df

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own,EPS next Y,Income,Perf Year,Perf YTD,Profit Margin,Payout,Sales
ELTK,2.73,14.58,9.61,NaN,0.09,0.70,NaN,19.5,9.4,146.8,25.30,1.07,4100000.0,143.60,149.86,10.0,NaN,NaN
OPCH,4.09,34.89,26.84,3.49,0.79,0.87,NaN,11.9,7.6,28.8,0.10,1.14,159500000.0,9.72,0.90,3.9,NaN,NaN
MELI,30.14,100.03,50.09,2.02,2.37,12.23,NaN,34.9,10.4,204.2,7.85,24.43,618000000.0,80.54,43.51,5.5,NaN,NaN
KOPN,5.21,NaN,NaN,NaN,0.00,-0.22,NaN,-61.9,-90.6,-64.3,0.90,-0.07,-20600000.0,80.40,64.40,-44.2,NaN,NaN
IMBIL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NTLA,3.36,NaN,NaN,NaN,0.00,-5.39,NaN,-41.7,-37.1,40.2,5.61,-5.76,-430400000.0,12.58,32.07,NaN,NaN,NaN
USEA,NaN,0.53,NaN,NaN,0.00,5.27,10.83,NaN,37.2,NaN,18.01,NaN,35100000.0,NaN,-15.34,NaN,NaN,NaN
MSSAU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
RFIL,1.14,82.91,9.66,NaN,0.36,0.06,NaN,1.4,3.6,-313.0,5.00,0.47,600000.0,-26.66,-9.53,0.6,NaN,NaN


# Filter Good Companies

### 1. Companies which are quoted at low valuations
P/E < 15 and P/B < 1

In [70]:
df_filtered = df[(df['P/E'].astype(float)<25) & (df['P/B'].astype(float) < 1)]
df_filtered = df_filtered[(df_filtered['Profit Margin'].astype(float) > 10)]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own,EPS next Y,Income,Perf Year,Perf YTD,Profit Margin,Payout,Sales
ACMR,0.99,14.28,11.45,0.33,0.11,0.80,NaN,7.8,5.6,211.7,1.80,1.00,52200000.0,-16.63,56.24,12.4,NaN,NaN
HBCP,0.83,7.06,7.84,NaN,0.17,5.03,2.81,12.4,26.1,163.5,0.20,4.54,41000000.0,2.06,-12.14,29.4,NaN,NaN
GURE,0.10,3.11,NaN,0.16,0.01,0.87,NaN,3.5,3.6,-367.0,3.70,NaN,9600000.0,-34.50,-12.25,14.5,NaN,NaN
PXS,0.59,2.07,NaN,NaN,0.86,1.87,NaN,42.1,13.1,150.0,53.90,NaN,24900000.0,32.04,-17.82,39.4,NaN,NaN
TRMK,0.88,14.82,9.85,2.12,0.12,1.51,4.10,6.0,27.9,73.7,0.30,2.28,93000000.0,-22.79,-36.35,14.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PBFS,0.86,11.25,NaN,NaN,0.00,0.79,NaN,8.0,16.4,1894.7,55.65,NaN,19800000.0,-14.60,-23.82,30.9,NaN,NaN
LARK,0.96,11.25,NaN,NaN,0.43,1.93,3.87,8.9,21.8,8.1,15.10,NaN,10100000.0,-8.19,-2.08,20.7,NaN,NaN
EGBN,0.58,6.20,8.76,0.85,0.06,3.75,7.75,9.7,21.8,-45.4,0.10,2.65,119400000.0,-50.71,-47.82,25.1,NaN,NaN
MLVF,0.80,17.25,15.08,NaN,0.17,0.91,NaN,4.7,16.0,8.8,3.00,1.04,6900000.0,-3.45,-11.72,17.8,NaN,NaN


### 2. Further filter companies which have demonstrated earning power 
EPS Q/Q > 10%

In [71]:
df_filtered = df_filtered[df_filtered['EPS next Y'].astype(float) > 5]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own,EPS next Y,Income,Perf Year,Perf YTD,Profit Margin,Payout,Sales
EDRY,0.35,1.91,2.11,NaN,0.59,7.37,NaN,19.7,17.5,-115.1,50.78,6.68,21500000.0,-35.87,-18.53,34.0,NaN,NaN
INDB,0.79,8.36,9.61,3.34,0.04,5.94,4.43,9.5,17.4,21.4,0.40,5.16,272000000.0,-39.28,-42.67,39.5,NaN,NaN
COOP,0.84,11.84,7.22,NaN,1.88,4.12,NaN,7.5,9.7,-93.9,0.80,6.76,302000000.0,23.87,20.07,17.3,NaN,NaN
ESEA,0.78,1.46,2.13,0.29,0.63,14.71,9.32,64.9,38.7,-0.7,34.80,10.07,105100000.0,-17.58,16.15,58.6,NaN,NaN
QCRH,0.92,7.58,7.70,0.76,0.35,5.83,0.54,13.4,19.8,7.6,0.40,5.74,102600000.0,-17.77,-12.93,30.6,NaN,NaN
ZION,0.90,4.96,5.65,NaN,0.14,5.85,5.66,18.8,42.2,4.6,1.00,5.13,873000000.0,-46.87,-42.59,28.5,NaN,NaN
PNFP,0.80,7.94,8.33,0.25,0.08,7.27,1.52,10.6,16.1,6.4,2.21,6.93,553700000.0,-21.89,-22.15,34.2,NaN,NaN
QFIN,0.96,4.99,3.23,1.89,0.01,3.28,4.40,20.6,19.7,-23.3,5.55,5.06,531400000.0,-1.42,-21.66,23.9,NaN,NaN


# Filtering for good debt ratio

In [76]:
df_filtered = df_filtered[df_filtered['Debt/Eq'].astype(float) < 0.8]
df_filtered

,P/B,P/E,Forward P/E,PEG,Debt/Eq,EPS (ttm),Dividend %,ROE,ROI,EPS Q/Q,Insider Own,EPS next Y,Income,Perf Year,Perf YTD,Profit Margin,Payout,Sales
EDRY,0.35,1.91,2.11,NaN,0.59,7.37,NaN,19.7,17.5,-115.1,50.78,6.68,21500000.0,-35.87,-18.53,34.0,NaN,NaN
INDB,0.79,8.36,9.61,3.34,0.04,5.94,4.43,9.5,17.4,21.4,0.40,5.16,272000000.0,-39.28,-42.67,39.5,NaN,NaN
ESEA,0.78,1.46,2.13,0.29,0.63,14.71,9.32,64.9,38.7,-0.7,34.80,10.07,105100000.0,-17.58,16.15,58.6,NaN,NaN
QCRH,0.92,7.58,7.70,0.76,0.35,5.83,0.54,13.4,19.8,7.6,0.40,5.74,102600000.0,-17.77,-12.93,30.6,NaN,NaN
ZION,0.90,4.96,5.65,NaN,0.14,5.85,5.66,18.8,42.2,4.6,1.00,5.13,873000000.0,-46.87,-42.59,28.5,NaN,NaN
PNFP,0.80,7.94,8.33,0.25,0.08,7.27,1.52,10.6,16.1,6.4,2.21,6.93,553700000.0,-21.89,-22.15,34.2,NaN,NaN
QFIN,0.96,4.99,3.23,1.89,0.01,3.28,4.40,20.6,19.7,-23.3,5.55,5.06,531400000.0,-1.42,-21.66,23.9,NaN,NaN


In [73]:
df_filtered.to_csv(output_directory+csv_name+"output.csv", index=True)